# Advent of Code 2023

## Contents
- [Day 1](#day-1)
- [Day 2](#day-2)
- [Day 3](#day-3)
- [Day 4](#day-4)

## Boilerplate

In [74]:
# SETUP #
import math

# TEST #
def test(test, solution):
    if test != solution:
        print(f"Test failed. Expected {solution}, but got {test}.")
    else: print("Test success!")

In [159]:
# FILE READING - Run *after* day config #
with open(f"inputs/{DAY}.txt", mode="rt") as f:
    PUZZLE_INPUT = f.read()

with open(f"test_inputs/{DAY}.txt", mode="rt") as f:
    TEST_INPUT = f.read()

## Day 1

The newly-improved calibration document consists of lines of text; each line originally contained a specific calibration value that the Elves now need to recover. On each line, the calibration value can be found by combining the first digit and the last digit (in that order) to form a single two-digit number.

For example:
```
1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet
```

In this example, the calibration values of these four lines are 12, 38, 15, and 77. Adding these together produces 142.

Consider your entire calibration document. What is the sum of all of the calibration values?

--- Part Two ---

Your calculation isn't quite right. It looks like some of the digits are actually spelled out with letters: one, two, three, four, five, six, seven, eight, and nine also count as valid "digits".

Equipped with this new information, you now need to find the real first and last digit on each line. For example:
```
two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
```

In this example, the calibration values are 29, 83, 13, 24, 42, 14, and 76. Adding these together produces 281.

What is the sum of all of the calibration values?

In [53]:
# DAY CONFIG #
DAY = "1"
TEST_SOLUTION_PART_1 = 142
TEST_SOLUTION_PART_2 = 281

In [59]:
# DAY 1 #
def run(part,i):
    
    # Part 2 only
    if part == 2:
        number_names = [
            ("one", "one1one"),
            ("two", "two2two"),
            ("three", "three3three"),
            ("four", "four4four"),
            ("five", "five5five"),
            ("six", "six6six"),
            ("seven", "seven7seven"),
            ("eight", "eight8eight"),
            ("nine", "nine9nine"),
        ]
        for pair in number_names:
            i = i.replace(pair[0], pair[1])

    total = 0
    # For each line, combine the first and last digits and add to total
    for line in i.splitlines():
        digits = ''.join(c for c in line if c.isdigit())
        total += int(digits[0] + digits[-1])
    return total

# Run test
test(run(1,"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 54951
Part 2: 55218


## Day 2

As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.

To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

For example, the record of a few games might look like this:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```

In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.

The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.

Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?

--- Part Two ---

As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?

Again consider the example games from earlier:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```

- In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
- Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
- Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
- Game 4 required at least 14 red, 3 green, and 15 blue cubes.
- Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.

The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum 2286.

For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

In [65]:
# DAY CONFIG #
DAY = "2"
TEST_SOLUTION_PART_1 = 8
TEST_SOLUTION_PART_2 = 2286

In [71]:
# DAY 2 #
def run(part,i):
    
    def clean_input(i):
        # Parse the input into the following format:
        # {'1': {'red': 4, 'green': 2, 'blue': 6}, '2': {'red': 1, 'green': 3, 'blue': 4}...
        clean = {}

        # Input line format -> Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
        for line in i.splitlines():
            # Break into "Game X" and game data (list of hands), then isolate game ID
            id, game = line.split(": ")
            id = id.split(" ")[1]
            
            clean[id] = {"red": 0, "green": 0, "blue": 0}

            for round in game.split("; "):
                for r in round.split(", "):
                    amount, colour = r.split(" ")
                    # Update the value for a colour only if it is higher than the existing value
                    clean[id][colour] = max(clean[id][colour], int(amount))

        return clean

    def validate(id,game):
        # Return game ID for valid games, 0 for invalid
        max_values = {"red": 12, "green": 13, "blue": 14}

        for colour, value in game.items():
            if value > max_values[colour]:
                return 0
            
        return int(id)

    def get_power(game):
        return math.prod(game.values())

    data = clean_input(i)
    
    total = 0
    total_power = 0

    for id, game in data.items():
        total += validate(id, game)
        total_power += get_power(game)

    return total if (part == 1) else total_power

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 2449
Part 2: 63981


## Day 3

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

--- Part Two ---

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [134]:
# DAY CONFIG #
DAY = "3"
TEST_SOLUTION_PART_1 = 4361
TEST_SOLUTION_PART_2 = 467835

In [155]:
# DAY 3 #
def run(part,i):
    # Convert input to 2D matrix
    i = list(map(list, i.splitlines()))

    checked = [[] for _ in i]
    parts = []
    gears = []

    def find_adjacent_numbers(row, column) -> ([int], int):
        adjacent_count = 0
        numbers = []
        gear_power = 0

        # Check row and column before and after
        for r in range(row-1, row+2):
            for c in range(column-1, column+2):
                if number := check_digit(r,c): 
                    numbers.append(number)
                    adjacent_count += 1

        if i[row][column] == '*' and adjacent_count == 2:
            gear_power = math.prod(numbers)
        
        return (numbers, gear_power)

    def check_digit(row, column) -> int:
        if row < 0 or row >= len(i) or column < 0 or column >= len(i[row]):
            return # out of bounds
        
        if i[row][column].isdigit() and column not in checked[row]:
            # Walk back to start of digit and save it
            number = ''
            while column > 0 and i[row][column-1].isdigit():
                column -= 1
                
            while column < len(i[row]) and i[row][column].isdigit():
                number += i[row][column]
                checked[row].append(column)
                column += 1

            return int(number)

    for row in range(len(i)):
        for column in range(len(i[row])):
            # Check for symbol, find numbers near it
            if not i[row][column].isdigit() and i[row][column] != '.':
                numbers, score = find_adjacent_numbers(row, column)
                parts.extend(numbers)
                gears.append(score)

    return sum(parts) if part == 1 else sum(gears)

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 556057
Part 2: 82824352


## Day 4

In [156]:
# DAY CONFIG #
DAY = "4"
TEST_SOLUTION_PART_1 = 0
TEST_SOLUTION_PART_2 = 0

In [158]:
# DAY 4 #
def run(part,i):
    return 0

# Run test
test(run(1,TEST_INPUT), TEST_SOLUTION_PART_1)
test(run(2,TEST_INPUT), TEST_SOLUTION_PART_2)

# Run on real input
print(f"Part 1: {run(1,PUZZLE_INPUT)}")
print(f"Part 2: {run(2,PUZZLE_INPUT)}")

Test success!
Test success!
Part 1: 0
Part 2: 0
